In [1]:
import pandas as pd
import requests
import gmaps
from config import g_key


In [2]:
gmaps.configure(api_key=g_key)

In [3]:
#load the city_weather.csv
file_to_load = 'Path/city_weather.csv'

In [4]:
file = pd.read_csv(file_to_load)

In [5]:
#create a new DataFrame
vacation_search_df = pd.DataFrame(file)
vacation_search_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,City Rain,City Snow,Weather Description,Date
0,0,Vaini,-21.20,-175.20,80.60,88,75,11.41,TO,0.0,0.0,light intensity shower rain,2020-02-09 21:06:44
1,1,Khandyga,62.67,135.60,-23.40,68,0,2.19,RU,0.0,0.0,clear sky,2020-02-09 21:06:45
2,2,Nizhniy Chir,48.36,43.09,8.73,87,0,4.79,RU,0.0,0.0,clear sky,2020-02-09 21:06:45
3,3,Hobart,-42.88,147.33,62.60,63,75,5.82,AU,0.0,0.0,broken clouds,2020-02-09 21:06:34
4,4,Busselton,-33.65,115.33,55.00,42,0,13.56,AU,0.0,0.0,clear sky,2020-02-09 21:06:45


In [6]:
#Filter the DataFrame for minimum and maximum temperature preferences
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 15
What is the maximum temperature you would like for your trip? 35


In [7]:
rain_choice = str(input("Do you want it to be raining? (yes/no) "))

Do you want it to be raining? (yes/no) no


In [8]:
snow_choice = str(input("Do you want it to be snowing? (yes/no) "))

Do you want it to be snowing? (yes/no) no


In [9]:
#set the 'Max Temp' to the prompted temp
vacation_search_df = vacation_search_df.loc[(vacation_search_df["Max Temp"] <= max_temp) & \
                                       (vacation_search_df["Max Temp"] >= min_temp)]
#set the precipitation to the selected option 
vacation_search_df = vacation_search_df.loc[(vacation_search_df['City Rain'] == 0) & \
                                            (vacation_search_df['City Snow'] == 0)]

In [10]:
#create a new column Hotel Name and set it to blank
vacation_df = vacation_search_df[["City", "Country", "Max Temp", "Lat", "Lng", "Weather Description"]].copy()
vacation_df["Hotel Name"] = ""

In [11]:
#count the number of cities
vacation_df.count()

City                   75
Country                75
Max Temp               75
Lat                    75
Lng                    75
Weather Description    75
Hotel Name             75
dtype: int64

In [14]:
#check for null values 
vacation_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Lat                    0
Lng                    0
Weather Description    0
Hotel Name             0
dtype: int64

In [15]:
vacation_df

,City,Country,Max Temp,Lat,Lng,Weather Description,Hotel Name
6,Fairbanks,US,25.00,64.84,-147.72,light snow,
12,Husavik,IS,28.40,66.04,-17.34,broken clouds,
32,Yarmolyntsi,UA,30.00,49.19,26.84,broken clouds,
37,Kodiak,US,33.80,57.79,-152.41,clear sky,
50,Albany,US,33.01,42.60,-73.97,overcast clouds,
...,...,...,...,...,...,...,...
522,Hasaki,JP,24.80,35.73,140.83,few clouds,
540,Morden,CA,23.00,49.19,-98.10,few clouds,
550,Bismil,TR,23.00,37.85,40.67,scattered clouds,
558,Spearfish,US,30.00,44.49,-103.86,clear sky,


In [16]:

params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
for index, row in vacation_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotels = requests.get(base_url, params=params).json()
    try:
        vacation_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [17]:
#Add the cities to a marker layer map with a pop-up marker for each city
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

In [18]:
locations = vacation_df[["Lat", "Lng"]]
max_temp = vacation_df["Max Temp"]
weather_desc = vacation_df["Weather Description"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [19]:
vacation_df.to_csv('WeatherPy_vacation.csv')